### dataset dir

In [1]:
import pandas as pd
import h5py
import xmltodict
import glob
import yaml
import tqdm
import pickle
import random
from os import listdir
from os.path import isfile, join, splitext

In [3]:
hf = h5py.File('/media/hdd1/fastMRIdata/brain/multicoil_train/file_brain_AXFLAIR_200_6002429.h5')
dic = xmltodict.parse(hf['ismrmrd_header'][()])['ismrmrdHeader']
dic['acquisitionSystemInformation']['systemModel']

'Skyra'

In [6]:
dic['acquisitionSystemInformation']['systemFieldStrength_T']

'2.8936'

In [10]:
hf['kspace'][()].shape[0]


16

TTT paper

In [26]:
import os
with open('/cheng/metaMRI/ttt_for_deep_learning_cs/unet/train_data/brain_val','rb') as fn:
    dataset = pickle.load(fn)

mypath = '/media/hdd1/fastMRIdata/brain/multicoil_train/'
filename_list = []

for i,d in enumerate(dataset):
    file_name = d['filename']
    filename, file_extension = os.path.splitext(file_name)
    filename_list.append(filename)

filename_list    

['file_brain_AXT2_200_2000501',
 'file_brain_AXT2_201_2010184',
 'file_brain_AXT2_209_6001308',
 'file_brain_AXT2_200_6002419',
 'file_brain_AXT2_200_2000590',
 'file_brain_AXT2_201_2010615',
 'file_brain_AXT2_201_2010082',
 'file_brain_AXT2_209_6001276',
 'file_brain_AXT2_201_2010304',
 'file_brain_AXT2_209_6001438',
 'file_brain_AXT2_201_2010253',
 'file_brain_AXT2_200_6002607',
 'file_brain_AXT2_200_2000069',
 'file_brain_AXT2_210_6001578',
 'file_brain_AXT2_200_2000405',
 'file_brain_AXT2_200_6002266',
 'file_brain_AXT2_209_2090034',
 'file_brain_AXT2_209_6001169',
 'file_brain_AXT2_201_2010074',
 'file_brain_AXT2_210_6001679',
 'file_brain_AXT2_210_6001501',
 'file_brain_AXT2_209_6001281',
 'file_brain_AXT2_200_2000264',
 'file_brain_AXT2_200_6002058',
 'file_brain_AXT2_200_2000402',
 'file_brain_AXT2_210_2100132',
 'file_brain_AXT2_210_2100190',
 'file_brain_AXT2_200_6002654',
 'file_brain_AXT2_200_6002509',
 'file_brain_AXT2_204_2040004',
 'file_brain_AXT2_201_2010246',
 'file_b

In [27]:
target_protocol = 'AXT2'    # AXT1, AXT2, AXFLAIR, AXT1POST, CORPD_FBK
target_FieldStrength = '2.8936'     

dataset_name = 'TTT_brain_test_all'

mypath = '/media/hdd1/fastMRIdata/brain/multicoil_val/'

# read file name in "mypath" into list
filenames = [splitext(filename)[0] for filename in listdir(mypath)]
slice_list = []

for filename in filenames:
    if filename in filename_list:
        pass
    else: 
        file_name = mypath + filename + '.h5'
        hf = h5py.File(file_name)
        dic = xmltodict.parse(hf['ismrmrd_header'][()])['ismrmrdHeader']
        # reconX = int(dic['encoding']['reconSpace']['matrixSize']['x'])
        # reconY = int(dic['encoding']['reconSpace']['matrixSize']['y'])
        kspace_shape = hf['kspace'].shape
        protocolName = dict(hf.attrs)['acquisition']
        FieldStrength = dic['acquisitionSystemInformation']['systemFieldStrength_T']
        if  protocolName == target_protocol and FieldStrength == target_FieldStrength\
            and kspace_shape[0] == 16: 
            slice_list.append({
                'path': mypath + filename+'.h5', 
                'slice': 8, 
                'filename': filename, 
                'predefined_mask': None})      
        
        with open("/cheng/metaMRI/metaMRI/data_dict_temp/" + dataset_name + ".yaml", "w") as fp:
            yaml.dump(slice_list,fp)
